This notebook covers scraping LGBTQ+ tagged articles from TVN. Due to technical difficulties, links to the articles were first collected in an Excel Spreadsheet. Following code goes through them and scrapes relevant information. It outputs a csv with articles' text, time and title.

In [6]:
#read news_links.xlsx file
news_links = pd.read_excel("news_links.xlsx")

In [9]:
news_links.head(5)
news_sample = news_links.sample(5)

In [10]:
news_sample

,url_name
10,https://tvn24.pl/polska/tvp-przegrywa-w-sadzie...
251,https://tvn24.pl/swiat/rosja-ruch-lgbt-uznany-...
130,https://tvn24.pl/swiat/wegry-referendum-w-spra...
96,https://tvn24.pl/swiat/prawa-osob-lgbt-w-ktory...
17,https://tvn24.pl/polska/jakub-kwiecinski-i-daw...


In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def main():
    # Assuming news_links is a DataFrame containing URLs
    urls = news_links["url_name"].tolist()  # Extract URLs from the DataFrame

    all_headers = []
    all_times = []
    all_descriptions = []
    all_texts = []

    headers = {
        "User-Agent": "Your User-Agent String"  # Add your user-agent string here
    }

    for url in urls:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            page_soup = BeautifulSoup(response.text, "html.parser")

            # Extract specific information from the page
            header = page_soup.find("h1", class_="heading heading--size-36 article-top-bar__title")
            header_text = header.get_text().strip() if header else "N/A"

            time = page_soup.find("time", class_="article-top-bar__date")
            time_text = time.get_text().strip() if time else "N/A"

            description = page_soup.find("p", class_="lead-text")
            description_text = description.get_text().strip() if description else "N/A"

            text = ' '.join(span.get_text().strip() for span in page_soup.find_all("p", class_="paragraph"))
            text = text if text else "N/A"

            all_headers.append(header_text)
            all_times.append(time_text)
            all_descriptions.append(description_text)
            all_texts.append(text)
        except Exception as e:
            print(f"Error scraping URL: {url}. {e}")

    data = {
        "URLs": urls,
        "Headers": all_headers,
        "Times": all_times,
        "Descriptions": all_descriptions,
        "Texts": all_texts,
    }

    df = pd.DataFrame.from_dict(data)
    df.to_csv("tvn24.csv", encoding="utf-8", header=True, index=False)

if __name__ == "__main__":
    main()


In [16]:
# Load tvn24 data
tvn24_df = pd.read_csv("tvn24.csv")
tvn24_df.head()

,URLs,Headers,Times,Descriptions,Texts
0,https://tvn24.pl/swiat/francja-brigitte-macron...,Brigitte Macron przeciwna formom neutralnym pł...,"15 grudnia 2022, 13:56","Brigitte Macron, pierwsza dama Francji, wyrazi...","Brigitte, żona prezydenta Francji Emmanuela Ma..."
1,https://tvn24.pl/polska/sejmowa-komisja-przeci...,Komisja przeciwko uchyleniu immunitetu Czarnko...,"14 grudnia 2022, 7:02",Sejmowa komisja regulaminowa odrzuciła wniosek...,"Sejmowa komisja Regulaminowa, Spraw Poselskich..."
2,https://tvn24.pl/swiat/usa-ustawa-uznajaca-mal...,Biden podpisał ustawę uznającą małżeństwa jedn...,"14 grudnia 2022, 5:24",Ustawa potwierdzająca federalne uznanie małżeń...,Prezydent USA Joe Biden podpisał we wtorek ust...
3,https://tvn24.pl/swiat/usa-ustawe-zapewniajaca...,Kongres przegłosował ustawę kodyfikującą prawo...,"9 grudnia 2022, 7:48",Izba Reprezentantów USA ostatecznie zatwierdzi...,Izba Reprezentantów przyjęła w czwartek ustawę...
4,https://tvn24.pl/swiat/usa-senat-przyjal-ustaw...,Senat USA przyjął ustawę o ochronie małżeństw ...,"30 listopada 2022, 5:47","Senat USA przyjął we wtorek ustawę, która chro...",Przyjęta przez amerykański Senat ustawa o ochr...
